In [9]:
import psycopg2
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')


## Establishing a connnection with a database

We actually need to establish a connection with our database vendor, in this case PostgreSQL, which will allow us to run queries against our database. Postgres is a bit of software that we need to find and connect to. If this is running locally (as in on our computers then we can use our machine's local IP address and just find the port that postgres is running on).

If we are running locally, our connection parameters will look something like the following:

```python
psycopg2.connect(database="databasename", 
                 user="postgres", 
                 password="somepassword",
                 host="127.0.0.1",
                 port="5432")
```

However, this information is sensitive and should never be exposed for security purposes. It is for this reason that this notebook is not hard-coding these values in. Instead, we are setting these values as part of the OS environment and reading them in.

In [6]:
dbname = os.environ.get("DB_NAME")
dbuser = os.environ.get("DB_USER")
dbpass = os.environ.get("DB_PASSWORD")
dbhost = os.environ.get("DB_HOST")
dbport = os.environ.get("DB_PORT")

conn = psycopg2.connect(database=dbname, 
                        user=dbuser, 
                        password=dbpass,
                        host=dbhost,
                        port=dbport)

### A sample query using `psycopg2`

When establishing and familiarizing yourself with a database, it is always best to perform the *hello, world* of SQL...

```sql
SELECT * 
FROM table_name;
```

In [8]:
# a cursor just allows us to execute queries against a database connection
cur = conn.cursor()
cur.execute("SELECT * FROM albums;") 
results = cur.fetchall()

# iterate through the list of tuples
for row in results: 
    print(row)

(1, 'Valentine', 'Snail Mail', datetime.date(2021, 11, 5), 'indie rock')
(2, 'Sweetener', 'Ariana Grande', datetime.date(2018, 8, 17), 'pop')
(3, 'To Pimp A Butterfly', 'Kendrick Lamar', datetime.date(2015, 3, 16), 'hip hop')
(4, 'Bury Me At Makeout Creek', 'Mitski', datetime.date(2014, 11, 11), 'indie rock')
(5, 'Blue', 'Joni Mitchell', datetime.date(1971, 6, 22), 'folk')
(6, 'channel ORANGE', 'Frank Ocean', datetime.date(2012, 7, 10), 'R&B')
(7, 'Changes', 'Justin Bieber', datetime.date(2020, 2, 14), 'pop')
(8, 'Illinoise', 'Sufjan Stevens', datetime.date(2005, 7, 4), 'folk')
(9, 'Sun Giant', 'Fleet Foxes', datetime.date(2008, 4, 8), 'folk')
(10, 'Sounds of Silence', 'Simon & Garfunkle', datetime.date(1966, 1, 17), 'folk')
(11, 'Lemonade', 'Beyoncé', datetime.date(2016, 4, 23), 'R&B')
(12, 'at last!', 'Etta James', datetime.date(1960, 11, 15), 'R&B')
(13, "I'm Wide Awake, It's Morning", 'Bright Eyes', datetime.date(2005, 1, 25), 'folk')
(14, 'Laurel Hell', 'Mitski', datetime.date(202

## Passing the connection to `pandas`

From here on out in this notebook we are going to pass the query and the connection to `pandas`'s `read_sql_query` method for formatting purposes. DataFrames just make the output a bit cleaner.

Let's try it out with the same query.

In [10]:
pd.read_sql_query("SELECT * FROM songs;", conn)

,id,album_id,track_number,song_name,rating,run_time
0,1,1,1,Valentine,8.0,0 days 00:03:16
1,2,1,2,Ben Franklin,8.5,0 days 00:03:02
2,3,1,3,Headlock,7.5,0 days 00:03:12
3,4,1,4,Light Blue,7.0,0 days 00:02:34
4,5,1,5,Forever (Sailing),8.0,0 days 00:04:18
...,...,...,...,...,...,...
181,181,15,6,He Doesn't Know Why,8.0,0 days 00:03:20
182,182,15,7,Heard Them Stirring,7.5,0 days 00:03:02
183,183,15,8,Your Protector,8.0,0 days 00:04:09
184,185,15,10,Blue Ridge Mountains,9.5,0 days 00:04:25


# Our Exercises for Today

### 1. List the total number of songs

In [11]:
query = """
SELECT COUNT(*) AS num_songs
FROM songs;
"""

pd.read_sql_query(query, conn)

,num_songs
0,186


### 2. List the songs with a rating over 9

In [12]:
query = """
SELECT *
FROM songs
WHERE rating > 9;
"""

pd.read_sql_query(query, conn)

,id,album_id,track_number,song_name,rating,run_time
0,31,3,6,u,9.5,0 days 00:04:28
1,32,3,7,Alright,9.5,0 days 00:03:39
2,59,5,8,River,10.0,0 days 00:04:04
3,63,6,2,Thinkin Bout You,9.5,0 days 00:03:20
4,68,6,7,Super Rich Kids,9.5,0 days 00:05:04
5,71,6,10,Pyramids,9.5,0 days 00:09:52
6,75,6,14,Bad Religion,9.5,0 days 00:02:55
7,76,6,15,Pink Matter,9.5,0 days 00:04:28
8,77,6,16,Forrest Gump,9.5,0 days 00:03:14
9,99,8,4,"John Wayne Gacy, Jr.",9.5,0 days 00:03:19


### 3. List the albums that have genre of "folk" arranged alphabetically by artist and then by release date newest to oldest.

In [13]:
query = """
SELECT * 
FROM albums
WHERE genre = 'folk'
ORDER BY artist_name, release_date DESC;
"""

pd.read_sql_query(query, conn)

,id,album_name,artist_name,release_date,genre
0,13,"I'm Wide Awake, It's Morning",Bright Eyes,2005-01-25,folk
1,15,Fleet Foxes,Fleet Foxes,2008-06-03,folk
2,9,Sun Giant,Fleet Foxes,2008-04-08,folk
3,5,Blue,Joni Mitchell,1971-06-22,folk
4,10,Sounds of Silence,Simon & Garfunkle,1966-01-17,folk
5,8,Illinoise,Sufjan Stevens,2005-07-04,folk


### 4. List all of the genres, don't repeat any.

In [14]:
query = """
SELECT DISTINCT genre
FROM albums
ORDER BY genre;
"""

pd.read_sql_query(query, conn)

,genre
0,folk
1,hip hop
2,indie rock
3,pop
4,R&B


In [ ]:
##